In [1]:
%pip install -r requirements.txt
from IPython.display import clear_output ; clear_output()

In [2]:
from util import initialize, show
AI_MODEL = initialize()

import asyncio
import json
from typing import List, Dict

from pydantic import BaseModel, Field
from pydantic_ai import Agent

Available AI models:
['openai:gpt-4o',
 'openai:gpt-4o-mini',
 'gemini-1.5-pro',
 'gemini-2.0-flash-exp',
 'claude-3-5-haiku-latest',
 'claude-3-5-sonnet-latest']
Using AI model: openai:gpt-4o


In [3]:
class Task(BaseModel):
    type: str = Field(..., description=(
        'The type of task. '
        'For example: "formal", "conversational", "hybrid", ...'))
    description: str = Field(..., description='Clear description for executing this task.')


class OrchestratorResponse(BaseModel):
    analysis: str = Field(..., description=(
        'Explain your understanding of the task and which variations would be valuable. '
        'Focus on how each approach serves different aspects of the task.'
    ))
    tasks: List[Task] = Field(..., description="List of tasks")


async def orchestrate(task: str) -> Dict:
    """Process task by breaking it down and running subtasks in parallel."""

    orchestrator_response = await Agent(
        AI_MODEL,
        system_prompt='Analyze this task and break it down into 2-3 distinct approaches.',
        result_type=OrchestratorResponse,
    ).run(task)
    
    # Parse orchestrator response
    analysis = orchestrator_response.data.analysis
    tasks = orchestrator_response.data.tasks
    
    show('', title='Orchestrator Output')
    show(analysis, title='Analysis')
    show('\n'.join([json.dumps(task.model_dump(), indent=2) for task in tasks]), title='Tasks')
    
    # Step 2: Process all the tasks in parallel and collect results
    worker_agent = Agent(AI_MODEL, system_prompt='Generate content based on the task specification.')
    worker_responses = await asyncio.gather(*[
        worker_agent.run(json.dumps({'original_task': task} | task_info.model_dump()))
        for task_info in tasks
    ])
    worker_results = [
        {
            'type': task.type,
            'description': task.description,
            'result': response.data,
        }
        for task, response in zip(tasks, worker_responses)
    ]

    for worker_result in worker_results:
        show(worker_result['result'], title=f"Worker Result ({worker_result['type']})")
    
    return {
        "analysis": analysis,
        "worker_results": worker_results,
    }

In [4]:
results = await orchestrate('Write a product description for a new eco-friendly water bottle.')


Orchestrator Output
-------------------


Analysis
--------

Writing a product description requires creativity and understanding of different customer needs. Here, we can approach this task in several ways, each focusing on different aspects or audiences for the eco-friendly water bottle. The three approaches I suggest are: a formal approach, a conversational approach, and a hybrid approach. Each will serve different purposes, covering a broad range of potential customer preferences.


Tasks
-----

{
  "type": "formal",
  "description": "Craft a product description focusing on the technical details, eco-friendly materials, and environmental impact of the water bottle. Highlight features such as BPA-free materials, recyclability, and any certifications."
}
{
  "type": "conversational",
  "description": "Write a product description in a friendly and engaging tone that speaks to a younger, eco-conscious audience. Use relatable language and emphasize lifestyle benefits, such as reducing p